In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

In [2]:
from typing import List
from pydantic import BaseModel, Field

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email
        
foo = User(name="Joe",age=32, email="joe@gmail.com")
foo.name

'Joe'

In [4]:
# incorrect type

foo = User(name="Joe",age="bar", email="joe@gmail.com")
foo.age

'bar'

In [5]:
class pUser(BaseModel):
    name: str
    age: int
    email: str
    
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")
foo_p.name

'Jane'

In [6]:
try:
    foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")
except Exception as e:
    print(e)

1 validation error for pUser
age
  value is not a valid integer (type=type_error.integer)


In [8]:
class Class(BaseModel):
    students: List[pUser]
    
obj = Class(
    students=[
        pUser(name="Jane", age=32, email="jane@gmail.com"),
        pUser(name="John", age=28, email="john@gmail.com")
    ]
)

obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com'), pUser(name='John', age=28, email='john@gmail.com')])

## Pydantic to OpenAI function definition


In [9]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [10]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'title': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'type': 'object',
  'properties': {'airport_code': {'title': 'Airport Code',
    'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [14]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")
    
try:
    weather_function1 = convert_pydantic_to_openai_function(WeatherSearch1)
except Exception as e:
    print(f"Error: {e}")

Error: 'description'


In [17]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str
        
try:
    weather_function2 = convert_pydantic_to_openai_function(WeatherSearch2)
    convert_pydantic_to_openai_function(WeatherSearch2)
    print(weather_function2)
except Exception as e:
    print(f"Error: {e}")

{'name': 'WeatherSearch2', 'description': 'Call this with an airport code to get the weather at that airport', 'parameters': {'title': 'WeatherSearch2', 'description': 'Call this with an airport code to get the weather at that airport', 'type': 'object', 'properties': {'airport_code': {'title': 'Airport Code', 'type': 'string'}}, 'required': ['airport_code']}}


In [18]:
from langchain.chat_models import AzureChatOpenAI

model = AzureChatOpenAI(temperature=0.3,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name="gpt-35-turbo",
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
)

model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [19]:
model_with_function = model.bind(functions=[weather_function])
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [20]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [21]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [23]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

chain = prompt | model_with_function

chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [24]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [25]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]
functions

[{'name': 'WeatherSearch',
  'description': 'Call this with an airport code to get the weather at that airport',
  'parameters': {'title': 'WeatherSearch',
   'description': 'Call this with an airport code to get the weather at that airport',
   'type': 'object',
   'properties': {'airport_code': {'title': 'Airport Code',
     'description': 'airport code to get weather for',
     'type': 'string'}},
   'required': ['airport_code']}},
 {'name': 'ArtistSearch',
  'description': 'Call this to get the names of songs by a particular artist',
  'parameters': {'title': 'ArtistSearch',
   'description': 'Call this to get the names of songs by a particular artist',
   'type': 'object',
   'properties': {'artist_name': {'title': 'Artist Name',
     'description': 'name of artist to look up',
     'type': 'string'},
    'n': {'title': 'N',
     'description': 'number of results',
     'type': 'integer'}},
   'required': ['artist_name', 'n']}}]

In [26]:
model_with_functions = model.bind(functions=functions)
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{\n  "airport_code": "SFO"\n}'}}, example=False)

In [27]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{\n  "artist_name": "taylor swift",\n  "n": 3\n}'}}, example=False)

In [28]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False)